<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/c/ca/Walmart_logo.svg/768px-Walmart_logo.svg.png" alt="WALMART LOGO" />

# **Walmart : predict weekly sales**

## **Company's Description** 📇

Walmart Inc. is an American multinational retail corporation that operates a chain of hypermarkets, discount department stores, and grocery stores from the United States, headquartered in Bentonville, Arkansas. The company was founded by Sam Walton in 1962.

## **Project** 🚧

Walmart's marketing service has requested to build a machine learning model able to estimate the weekly sales in their stores, with the best precision possible on the predictions made. Such a model would help them understand better how the sales are influenced by economic indicators, and might be used to plan future marketing campaigns.

## **Goals** 🎯

The project can be divided into three steps:

- Part 1: EDA and all the necessary preprocessings to prepare data for machine learning
- Part 2: train a **linear regression model** (baseline)
- Part 3: avoid overfitting by training a **regularized regression model**
- Part 4: fine tuning the regularization strenght
- Part 5: introduction of weights

## **Scope of this project** 🖼️

For this project, I'll work with a dataset that contains information about weekly sales achieved by different Walmart stores, and other variables such as the unemployment rate or the fuel price, that might be useful for predicting the amount of sales. The dataset covers sales from 2010-02-05 to 2012-11-01, in the file Walmart_Store_sales. Within this file you will find the following fields:

*   Store - the store number
*   Date - the week of sales
*   Weekly_Sales - sales for the given store
*   Holiday_Flag - whether the week is a special holiday week: 1 – Holiday week 0 – Non-holiday week
*   Temperature - Temperature on the day of sale
*   Fuel_Price - Cost of fuel in the region
*   CPI – Prevailing consumer price index
*   Unemployment - Prevailing unemployment rate

## **Part 1: EDA and preprocessing**


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error, r2_score
import warnings

warnings.filterwarnings(
    "ignore", category=DeprecationWarning
)  # to avoid deprecation warnings

In [ ]:
sales_file = "/content/drive/MyDrive/Colab Notebooks/Walmart_project/Walmart_Store_sales.csv" # Path to CSV file

# Load file
sales_df = pd.read_csv(sales_file)

### **1. EDA**

In [ ]:
# Display the first few rows of the DataFrame
sales_df.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6.0,18-02-2011,1572117.54,NaN,59.61,3.045,214.777523,6.858
1,13.0,25-03-2011,1807545.43,0.0,42.38,3.435,128.616064,7.470
2,17.0,27-07-2012,NaN,0.0,NaN,NaN,130.719581,5.936
3,11.0,NaN,1244390.03,0.0,84.57,NaN,214.556497,7.346
4,6.0,28-05-2010,1644470.66,0.0,78.89,2.759,212.412888,7.092


In [ ]:
# Display the shape of the dataframe (rows, columns)
sales_df.shape

(150, 8)

- There are only 150 rows and 8 features.

In [ ]:
# Display summary statistics of the DataFrame
sales_df.describe(include="all")

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,150.000000,132,1.360000e+02,138.000000,132.000000,136.000000,138.000000,135.000000
unique,NaN,85,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,19-10-2012,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN
mean,9.866667,NaN,1.249536e+06,0.079710,61.398106,3.320853,179.898509,7.598430
std,6.231191,NaN,6.474630e+05,0.271831,18.378901,0.478149,40.274956,1.577173
min,1.000000,NaN,2.689290e+05,0.000000,18.790000,2.514000,126.111903,5.143000
25%,4.000000,NaN,6.050757e+05,0.000000,45.587500,2.852250,131.970831,6.597500
50%,9.000000,NaN,1.261424e+06,0.000000,62.985000,3.451000,197.908893,7.470000
75%,15.750000,NaN,1.806386e+06,0.000000,76.345000,3.706250,214.934616,8.150000


- Data were collected from 20 stores. There are missing values for all features ( except for 'store').



In [ ]:
# Display information about the DataFrame
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         150 non-null    float64
 1   Date          132 non-null    object 
 2   Weekly_Sales  136 non-null    float64
 3   Holiday_Flag  138 non-null    float64
 4   Temperature   132 non-null    float64
 5   Fuel_Price    136 non-null    float64
 6   CPI           138 non-null    float64
 7   Unemployment  135 non-null    float64
dtypes: float64(7), object(1)
memory usage: 9.5+ KB


- 'Date' is an object and needs to be converted.

In [ ]:
# Display missing values
sales_df.isnull().sum()

,0
Store,0
Date,18
Weekly_Sales,14
Holiday_Flag,12
Temperature,18
Fuel_Price,14
CPI,12
Unemployment,15


- As observed before almost all the features have missing values. Let's analyze feature by feature:

#### **1.0. Stores**

In [ ]:
# Count rows per store
rows_per_store = sales_df.groupby('Store').size()
print(rows_per_store)

Store
1.0     11
2.0      8
3.0     15
4.0      7
5.0      9
6.0      7
7.0      8
8.0      8
9.0      4
10.0     5
11.0     3
12.0     5
13.0     9
14.0     9
15.0     4
16.0     4
17.0     8
18.0    10
19.0     9
20.0     7
dtype: int64


- The information available is different from a store to another. Some stores have only 4 entries while store 3 for example has 15 entries. This can bias the analysis.

In [ ]:
# Display rows per store
for i in range(1,16):
  store_i_sales = sales_df[sales_df['Store'] == i]
  print(store_i_sales)

     Store        Date  Weekly_Sales  Holiday_Flag  Temperature  Fuel_Price  \
13     1.0  16-03-2012    1677472.78           0.0        64.74       3.734   
42     1.0         NaN    1661767.33           1.0          NaN       3.730   
44     1.0  12-02-2010    1641957.44           1.0        38.51       2.548   
48     1.0  05-08-2011    1624383.75           NaN        91.65       3.684   
73     1.0  27-08-2010    1449142.92           NaN        85.22       2.619   
74     1.0  19-10-2012    1508068.77           0.0        67.97       3.594   
78     1.0  18-11-2011    1539483.70           0.0        62.25       3.308   
95     1.0  14-05-2010    1494251.50           0.0        74.78       2.854   
109    1.0  09-12-2011           NaN           NaN        43.93       3.158   
123    1.0  27-05-2011    1466046.67           0.0          NaN       3.786   
133    1.0  24-09-2010           NaN           0.0        80.94         NaN   

            CPI  Unemployment  
13   221.211813    

- Since the dates don't match from a store to another and since they are not successive, it's impossible to figure out the missing values.

#### **1.1. Weekly_Sales (Target)**

In [ ]:
# Create a boxplot of 'Weekly_Sales'
fig = px.box(sales_df, y="Weekly_Sales", title="Boxplot of Weekly_Sales (Dollar)")
fig.show()

In [ ]:
# Create a histogram of 'Weekly_Sales'
fig = px.histogram(sales_df, x="Weekly_Sales", title="Distribution of Weekly_Sales (Dollar)")
fig.show()

In [ ]:
# Drop rows where Weekly_Sales is missing
sales_df.dropna(subset=['Weekly_Sales'], inplace=True)

- Since 'Weekly_Sales' is the Target, rows with missing values in 'Weekly_Sales' were dropped.

#### **1.2. Date**

In [ ]:
sales_df.dropna(subset=['Date'], inplace=True)
sales_df['Date'] = pd.to_datetime(sales_df['Date'], format='%d-%m-%Y')
sales_df['Year'] = sales_df['Date'].dt.year
sales_df['Month'] = sales_df['Date'].dt.month
sales_df['Week'] = sales_df['Date'].dt.isocalendar().week
sales_df['Day'] = sales_df['Date'].dt.day
sales_df['DayOfWeek'] = sales_df['Date'].dt.dayofweek
sales_df = sales_df.drop('Date', axis=1)

- Rows with missing dates were dropped. Then 'Date' was converted from object to a date format and year, month, week, day and day of week were extracted. Finally 'Date' was dropped.

In [ ]:
sales_df.head()

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Week,Day,DayOfWeek
0,6.0,1572117.54,NaN,59.61,3.045,214.777523,6.858,2011,2,7,18,4
1,13.0,1807545.43,0.0,42.38,3.435,128.616064,7.470,2011,3,12,25,4
4,6.0,1644470.66,0.0,78.89,2.759,212.412888,7.092,2010,5,21,28,4
5,4.0,1857533.70,0.0,NaN,2.756,126.160226,7.896,2010,5,21,28,4
6,15.0,695396.19,0.0,69.80,4.069,134.855161,7.658,2011,6,22,3,4


- Since 'DayOfWeek' is always the same, it will be dropped.

#### **1.3. Holiday_Flag**

In [ ]:
holiday_flag_counts = sales_df['Holiday_Flag'].value_counts(dropna=False)
print(holiday_flag_counts)

Holiday_Flag
0.0    101
NaN      9
1.0      8
Name: count, dtype: int64


- There are 9 misssing 'Holiday_Flag', let's try to replace them using information from other rows:

In [ ]:
# Function to identify and replace missing Holiday_Flag values
def fill_missing_holiday_flags(df):
    """Identifies and replaces missing Holiday_Flag values based on matching weeks.

    Args:
        df: pandas DataFrame containing the data.

    Returns:
        pandas DataFrame with missing Holiday_Flag values filled.
    """

    # 1. Group by Week and get row counts
    rows_per_week = df.groupby('Week').size()

    # 2. Filter for missing Holiday_Flag
    missing_holiday_flag = df[df['Holiday_Flag'].isnull()]

    # 3. Compare and identify matches
    weeks_with_missing_flag = missing_holiday_flag['Week'].unique()

    # 4. Replace missing values based on matching weeks
    for week in weeks_with_missing_flag:
        # Get Holiday_Flag value(s) for the week
        holiday_flags_in_week = df[df['Week'] == week]['Holiday_Flag'].dropna().unique()

        # If there's only one unique Holiday_Flag value for the week
        if len(holiday_flags_in_week) == 1:
            # Fill missing values for that week with the unique value
            df.loc[(df['Week'] == week) & (df['Holiday_Flag'].isnull()), 'Holiday_Flag'] = holiday_flags_in_week[0]
        else:
            # Handle cases with multiple Holiday_Flag values (e.g., using mode or other logic)
            # For now, we'll just print a message
            print(f"Week {week} has multiple Holiday_Flag values: {holiday_flags_in_week}")
            print("Consider handling these cases separately.")

    return df

# Apply the function to your DataFrame
sales_df = fill_missing_holiday_flags(sales_df)

In [ ]:
holiday_flag_counts = sales_df['Holiday_Flag'].value_counts(dropna=False)
print(holiday_flag_counts)

Holiday_Flag
0.0    110
1.0      8
Name: count, dtype: int64


- There is no more missing values.

#### **1.4. Temperature**

In [ ]:
# Convert 'Temperature' from Fahrenheit to Celsius
sales_df['Temperature'] = (sales_df['Temperature'] - 32) * 5/9
sales_df.head()

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Week,Day,DayOfWeek
0,6.0,1572117.54,0.0,15.338889,3.045,214.777523,6.858,2011,2,7,18,4
1,13.0,1807545.43,0.0,5.766667,3.435,128.616064,7.470,2011,3,12,25,4
4,6.0,1644470.66,0.0,26.050000,2.759,212.412888,7.092,2010,5,21,28,4
5,4.0,1857533.70,0.0,NaN,2.756,126.160226,7.896,2010,5,21,28,4
6,15.0,695396.19,0.0,21.000000,4.069,134.855161,7.658,2011,6,22,3,4


In [ ]:
# Create a boxplot of 'Temperature'
fig = px.box(sales_df, y="Temperature", title="Boxplot of Temperature (Celsius)")
fig.show()

In [ ]:
# Create a histogram of 'Temperature'
fig = px.histogram(sales_df, x="Temperature", title="Distribution of Temperature (Celsius)")
fig.show()

#### **1.5. Fuel_Price**

In [ ]:
# Create a boxplot of 'Fuel_Price'
fig = px.box(sales_df, y="Fuel_Price", title="Boxplot of Fuel_price (Dollar)")
fig.show()

In [ ]:
# Create a histogram of 'Fuel_Price'
fig = px.histogram(sales_df, x="Fuel_Price", title="Distribution of Fuel_Price (Dollar)")
fig.show()

#### **1.6. CPI**

In [ ]:
# Create a boxplot of 'CPI'
fig = px.box(sales_df, y="CPI", title="Boxplot of Prevailing Consumer Price Index")
fig.show()

In [ ]:
# Create a histogram of 'CPI'
fig = px.histogram(sales_df, x="CPI", title="Distribution of CPI")
fig.show()

#### **1.7. Unemployment**

In [ ]:
# Create a boxplot of 'Unemployment'
fig = px.box(sales_df, y="Unemployment", title="Boxplot of Unemployment")
fig.show()

In [ ]:
# Create a histogram of 'Unemployment'
fig = px.histogram(sales_df, x="Unemployment", title="Distribution of Unemployment")
fig.show()

#### **1.8. Drop outliers from Unemployment, Temperature, Fuel_Price, CPI**

In [ ]:
# Features to remove outliers from
features_to_clean = ['Unemployment', 'Temperature', 'Fuel_Price', 'CPI']

for feature in features_to_clean:
    # Calculate the mean and standard deviation
    mean = sales_df[feature].mean()
    std = sales_df[feature].std()

    # Define the lower and upper bounds
    lower_bound = mean - 3 * std
    upper_bound = mean + 3 * std

    # Get the initial number of rows
    initial_rows = len(sales_df)

    # Filter the DataFrame to remove outliers
    sales_df = sales_df[(sales_df[feature] >= lower_bound) & (sales_df[feature] <= upper_bound)]

    # Calculate and print the number of dropped rows
    dropped_rows = initial_rows - len(sales_df)
    print(f"Number of rows dropped for '{feature}': {dropped_rows}")

Number of rows dropped for 'Unemployment': 16
Number of rows dropped for 'Temperature': 8
Number of rows dropped for 'Fuel_Price': 10
Number of rows dropped for 'CPI': 4


- After dropping outliers, there is no more missing values:

In [ ]:
sales_df.isnull().sum()

,0
Store,0
Weekly_Sales,0
Holiday_Flag,0
Temperature,0
Fuel_Price,0
CPI,0
Unemployment,0
Year,0
Month,0
Week,0


In [ ]:
sales_df.shape

(80, 11)

### **2. Preprocessing**

#### **2.0. Drop useless column**

In [ ]:
# Drop useless columns
useless_cols = ["DayOfWeek"]

print("Dropping useless columns...")
sales_df = sales_df.drop(
    useless_cols, axis=1
)
print("...Done.")
print(sales_df.head())

Dropping useless columns...
...Done.
   Store  Weekly_Sales  Holiday_Flag  Temperature  Fuel_Price         CPI  \
0    6.0    1572117.54           0.0    15.338889       3.045  214.777523   
1   13.0    1807545.43           0.0     5.766667       3.435  128.616064   
4    6.0    1644470.66           0.0    26.050000       2.759  212.412888   
6   15.0     695396.19           0.0    21.000000       4.069  134.855161   
7   20.0    2203523.20           0.0     4.405556       3.617  213.023622   

   Unemployment  Year  Month  Week  Day  
0         6.858  2011      2     7   18  
1         7.470  2011      3    12   25  
4         7.092  2010      5    21   28  
6         7.658  2011      6    22    3  
7         6.961  2012      2     5    3  


#### **2.1. Test Train Split**

In [ ]:
# Separate target variable Y from features X
target_name = 'Weekly_Sales'

print("Separating labels from features...")
Y = sales_df.loc[:, target_name]
X = sales_df.drop(target_name, axis=1)  # All columns are kept, except the target
print("...Done.")
print(Y.head())
print()
print(X.head())
print()

Separating labels from features...
...Done.
0    1572117.54
1    1807545.43
4    1644470.66
6     695396.19
7    2203523.20
Name: Weekly_Sales, dtype: float64

   Store  Holiday_Flag  Temperature  Fuel_Price         CPI  Unemployment  \
0    6.0           0.0    15.338889       3.045  214.777523         6.858   
1   13.0           0.0     5.766667       3.435  128.616064         7.470   
4    6.0           0.0    26.050000       2.759  212.412888         7.092   
6   15.0           0.0    21.000000       4.069  134.855161         7.658   
7   20.0           0.0     4.405556       3.617  213.023622         6.961   

   Year  Month  Week  Day  
0  2011      2     7   18  
1  2011      3    12   25  
4  2010      5    21   28  
6  2011      6    22    3  
7  2012      2     5    3  



In [ ]:
# First : always divide dataset into train set & test set !!
print("Dividing into train and test sets...")
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
# test_size indicates the proportion of rows from X and Y that will go into the test dataset while
# maintaining the correspondance between the rows from X and Y

# random_state is an argument that can be found in all functions that have a pseudo-random behaviour
# if random_state is not stated the function will derive a different random result everytime the cell
# runs, if random_state is given a value the results will be the same everytime the cell runs
print("...Done.")
print()

Dividing into train and test sets...
...Done.



#### **2.2. Standardizing and one-hot encoding**

In [ ]:
# Create pipeline for numeric features
numeric_features = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Year', 'Month', 'Week', 'Day']  # Names of numeric columns in X_train/X_test
numeric_transformer = Pipeline(
    steps=[
        ('scaler', StandardScaler())
    ]
)

In [ ]:
# Create pipeline for categorical features
categorical_features = ['Store', 'Holiday_Flag']  # Names of categorical columns in X_train/X_test
categorical_transformer = Pipeline(
    steps=[
        (
            'encoder',
            OneHotEncoder(drop="first"),
        ),  # first column will be dropped to avoid creating correlations between features
    ]
)

In [ ]:
# Use ColumnTransformer to make a preprocessor object that describes all the treatments to be done
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
    ]
)

In [ ]:
# Preprocessings on train set
print("Performing preprocessings on train set...")
print(X_train.head())
X_train = preprocessor.fit_transform(X_train)
print("...Done.")
print(
    X_train[0:5]
)  # MUST use this syntax because X_train is a numpy array and not a pandas DataFrame anymore
print()

# Preprocessings on test set
print("Performing preprocessings on test set...")
print(X_test.head())
X_test = preprocessor.transform(
    X_test
)  # We don't fit again !! The test set is used for validating decisions
# we made based on the training set, therefore we can only apply transformations that were parametered using the training set.
# Otherwise this creates what is called a leak from the test set which will introduce a bias in all the results.
print("...Done.")
print(
    X_test[0:5, :]
)  # MUST use this syntax because X_test is a numpy array and not a pandas DataFrame anymore
print()

Performing preprocessings on train set...
    Store  Holiday_Flag  Temperature  Fuel_Price         CPI  Unemployment  \
6    15.0           0.0    21.000000       4.069  134.855161         7.658   
38    4.0           0.0    27.694444       3.570  129.066300         5.946   
12    7.0           0.0     2.561111       3.767  192.826069         8.595   
44    1.0           1.0     3.616667       2.548  211.242170         8.106   
66   18.0           0.0    23.150000       2.792  132.614193         9.342   

    Year  Month  Week  Day  
6   2011      6    22    3  
38  2011      6    25   24  
12  2011      5    19   13  
44  2010      2     6   12  
66  2010      8    31    6  
...Done.
[[ 0.58589224  1.56199819 -1.05536176  0.30814094  0.18751465 -0.1099386
  -0.25016195 -1.45366334  0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          1.          0.          0.          0.
   0.          0.          0.      

## **Part 2: Train a linear regression model (baseline)**


### **1. Train Model**

In [ ]:
# Train model
model = LinearRegression()

print("Training model...")
model.fit(X_train, Y_train)  # Training is always done on train set !!
print("...Done.")

Training model...
...Done.


In [ ]:
# Predictions on training set
print("Predictions on training set...")
Y_train_pred = model.predict(X_train)
print("...Done.")
print(Y_train_pred[0:5])
print()

Predictions on training set...
...Done.
[ 662731.0782642  2062687.62195357  485854.24778584 1580481.34029635
 1095106.81558648]



In [ ]:
# Predictions on test set
print("Predictions on test set...")
Y_test_pred = model.predict(X_test)
print("...Done.")
print(Y_test_pred[0:5])
print()

Predictions on test set...
...Done.
[1692717.64233541 1854074.3351078  1538969.09218071  661668.5301425
 1585600.7840879 ]



### **2. Evaluate Model**


In [ ]:
# Print scores using appropriate metrics for regression
print("R-squared on training set : ", round(r2_score(Y_train, Y_train_pred), 2))
print("R-squared on test set : ", round(r2_score(Y_test, Y_test_pred), 2))

mse_train = mean_squared_error(Y_train, Y_train_pred)
mse_test = mean_squared_error(Y_test, Y_test_pred)

print("Mean Squared Error on training set : ", round(mse_train, 2))
print("Root Mean Squared Error on training set : ", round(np.sqrt(mse_train), 2)) # Calculate and print RMSE for training set

print("Mean Squared Error on test set : ", round(mse_test, 2))
print("Root Mean Squared Error on test set : ", round(np.sqrt(mse_test), 2)) # Calculate and print RMSE for test set

R-squared on training set :  0.98
R-squared on test set :  0.96
Mean Squared Error on training set :  7434352577.49
Root Mean Squared Error on training set :  86222.69
Mean Squared Error on test set :  19419091353.75
Root Mean Squared Error on test set :  139352.4


- There is a clear overfitting. Also, the test set MSE is higher than the training set.

In [ ]:
# Get feature names after preprocessing
feature_names = numeric_features + list(preprocessor.named_transformers_['cat']['encoder'].get_feature_names_out(categorical_features))

# Get coefficients
coefficients = model.coef_

# Create a DataFrame to display coefficients with feature names
coefficients_df = pd.DataFrame({'Feature': feature_names, 'Coefficient': coefficients})

# Sort coefficients by absolute value to see most impactful features
coefficients_df['Absolute_Coefficient'] = coefficients_df['Coefficient'].abs()
coefficients_df = coefficients_df.sort_values(by='Absolute_Coefficient', ascending=False)

# Display the coefficients
print(coefficients_df)

             Feature   Coefficient  Absolute_Coefficient
10         Store_4.0  4.052564e+06          4.052564e+06
16        Store_10.0  3.986540e+06          3.986540e+06
18        Store_13.0  3.943969e+06          3.943969e+06
24        Store_19.0  3.207406e+06          3.207406e+06
23        Store_18.0  2.875819e+06          2.875819e+06
22        Store_17.0  2.600460e+06          2.600460e+06
20        Store_15.0  2.507119e+06          2.507119e+06
19        Store_14.0  1.805508e+06          1.805508e+06
2                CPI  1.644945e+06          1.644945e+06
11         Store_5.0 -1.497880e+06          1.497880e+06
9          Store_3.0 -1.459814e+06          1.459814e+06
15         Store_9.0 -1.455437e+06          1.455437e+06
14         Store_8.0 -1.001785e+06          1.001785e+06
25        Store_20.0  6.197518e+05          6.197518e+05
21        Store_16.0 -2.997000e+05          2.997000e+05
6               Week  2.933174e+05          2.933174e+05
5              Month -2.545529e

In [ ]:
# Create bar plot
fig = px.bar(
    coefficients_df,
    x="Feature",
    y="Coefficient",
    title="Feature Coefficients of Linear Regression Model",
)
fig.update_xaxes(tickangle=45)  # Rotate x-axis labels for readability
fig.show()

In [ ]:
# Get the intercept (bias term)
intercept = model.intercept_

# Create the equation string
equation = f"Weekly_Sales = {intercept:.2f}"  # Start with the intercept

# Add terms for each feature with non-zero coefficients
for index, row in coefficients_df[coefficients_df['Coefficient'] != 0].iterrows():
    feature_name = row['Feature']
    coefficient = row['Coefficient']
    equation += f" + {coefficient:.2f} * {feature_name}"

# Print the equation
print("Linear Regression Equation:")
print(equation)

Linear Regression Equation:
Weekly_Sales = 38826.46 + 4052564.12 * Store_4.0 + 3986540.22 * Store_10.0 + 3943969.06 * Store_13.0 + 3207405.56 * Store_19.0 + 2875819.48 * Store_18.0 + 2600459.65 * Store_17.0 + 2507118.68 * Store_15.0 + 1805507.65 * Store_14.0 + 1644944.53 * CPI + -1497880.29 * Store_5.0 + -1459814.20 * Store_3.0 + -1455437.01 * Store_9.0 + -1001785.20 * Store_8.0 + 619751.81 * Store_20.0 + -299700.02 * Store_16.0 + 293317.43 * Week + -254552.93 * Month + 176779.86 * Store_2.0 + -133809.87 * Holiday_Flag_1.0 + -115176.61 * Store_7.0 + -103274.61 * Year + -93978.30 * Unemployment + -81861.58 * Fuel_Price + -74125.33 * Day + -69179.16 * Store_6.0 + 57779.87 * Store_11.0 + -56960.00 * Temperature


- The store is the most impactful feature on the weekly sales which is expected. It's followed by CPI.

##**Part 3: Train regularized regression models**




### **1. Train Lasso Model**

In [ ]:
# Train Lasso model
lasso_model = Lasso(alpha=0.1)  # The alpha value (regularization strength) can be adjusted (later)

print("Training Lasso model...")
lasso_model.fit(X_train, Y_train)
print("...Done.")

# Predictions on training set
print("Predictions on training set...")
Y_train_pred_lasso = lasso_model.predict(X_train)
print("...Done.")
print(Y_train_pred_lasso[0:5])
print()

# Predictions on test set
print("Predictions on test set...")
Y_test_pred_lasso = lasso_model.predict(X_test)
print("...Done.")
print(Y_test_pred_lasso[0:5])
print()

# Print scores using appropriate metrics for regression
print("R-squared on training set (Lasso): ", round(r2_score(Y_train, Y_train_pred_lasso), 2))
print("R-squared on test set (Lasso): ", round(r2_score(Y_test, Y_test_pred_lasso), 2))

mse_train_lasso = mean_squared_error(Y_train, Y_train_pred_lasso)
mse_test_lasso = mean_squared_error(Y_test, Y_test_pred_lasso)

print("Mean Squared Error on training set : ", round(mse_train_lasso, 2))
print("Root Mean Squared Error on training set : ", round(np.sqrt(mse_train_lasso), 2)) # Calculate and print RMSE for training set

print("Mean Squared Error on test set : ", round(mse_test_lasso, 2))
print("Root Mean Squared Error on test set : ", round(np.sqrt(mse_test_lasso), 2)) # Calculate and print RMSE for test set

Training Lasso model...
...Done.
Predictions on training set...
...Done.
[ 636559.95640835 2066222.81272147  500589.52865009 1591203.19723107
 1079523.78864314]

Predictions on test set...
...Done.
[1804382.38693231 1767288.48566788 1572344.63307986  544314.88483521
 1620019.12899912]

R-squared on training set (Lasso):  0.98
R-squared on test set (Lasso):  0.97
Mean Squared Error on training set :  8177913205.69
Root Mean Squared Error on training set :  90431.82
Mean Squared Error on test set :  12271781180.54
Root Mean Squared Error on test set :  110778.07


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.616e+11, tolerance: 2.905e+09



- The overfitting is improved by still there.

In [ ]:
# Get feature names after preprocessing
feature_names = numeric_features + list(preprocessor.named_transformers_['cat']['encoder'].get_feature_names_out(categorical_features))

# Get coefficients
coefficients_lasso = lasso_model.coef_

# Create a DataFrame to display coefficients with feature names
coefficients_df_lasso = pd.DataFrame({'Feature': feature_names, 'Coefficient': coefficients_lasso})

# Sort coefficients by absolute value to see most impactful features
coefficients_df_lasso['Absolute_Coefficient'] = coefficients_df_lasso['Coefficient'].abs()
coefficients_df_lasso = coefficients_df_lasso.sort_values(by='Absolute_Coefficient', ascending=False)


# Create bar plot using Plotly
fig = px.bar(
    coefficients_df_lasso,
    x="Feature",
    y="Coefficient",
    title="Feature Coefficients of Lasso Regression Model",
)
fig.update_xaxes(tickangle=45)  # Rotate x-axis labels for readability
fig.show()

- The ranking has also changed!

In [ ]:
# Get the intercept (bias term)
intercept = lasso_model.intercept_

# Create the equation string
equation = f"Weekly_Sales = {intercept:.2f}"  # Start with the intercept

# Add terms for each feature with non-zero coefficients
for index, row in coefficients_df_lasso[coefficients_df_lasso['Coefficient'] != 0].iterrows():
    feature_name = row['Feature']
    coefficient = row['Coefficient']
    equation += f" + {coefficient:.2f} * {feature_name}"

# Print the equation
print("Lasso Regression Equation:")
print(equation)

Lasso Regression Equation:
Weekly_Sales = 1365416.31 + -1436525.05 * Store_5.0 + -1328149.05 * Store_3.0 + -1324853.53 * Store_9.0 + 1165726.09 * Store_4.0 + -1019304.95 * Store_16.0 + 1015314.21 * Store_13.0 + 964565.80 * Store_10.0 + -894444.78 * Store_7.0 + -870106.85 * Store_8.0 + 771847.70 * Store_14.0 + 471088.86 * Store_19.0 + 384193.96 * Store_20.0 + 326991.88 * CPI + -302912.52 * Store_17.0 + -248782.76 * Store_15.0 + 145479.42 * Store_11.0 + -137891.77 * Holiday_Flag_1.0 + 127514.98 * Store_2.0 + 113583.96 * Store_18.0 + -97245.36 * Fuel_Price + 74182.21 * Month + -66538.80 * Unemployment + -53769.03 * Temperature + -45547.16 * Day + -36901.34 * Store_6.0 + 30145.50 * Year + -20854.20 * Week


In [ ]:

# Check for dropped features (coefficients equal to zero)
dropped_features = coefficients_df_lasso[coefficients_df_lasso['Coefficient'] == 0]['Feature'].tolist()

if dropped_features:
    print("Features dropped by Lasso:")
    for feature in dropped_features:
        print(feature)
else:
    print("Lasso did not drop any features.")

Lasso did not drop any features.


- The 'DayOfWeek' feature was alreday dropped.

### **2. Train Ridge Model**

In [ ]:
# Train Ridge model
ridge_model = Ridge(alpha=0.1)  # The alpha value (regularization strength) can be adjusted (later)

print("Training Lasso model...")
ridge_model.fit(X_train, Y_train)
print("...Done.")

# Predictions on training set
print("Predictions on training set...")
Y_train_pred_ridge = ridge_model.predict(X_train)
print("...Done.")
print(Y_train_pred_ridge[0:5])
print()

# Predictions on test set
print("Predictions on test set...")
Y_test_pred_ridge = ridge_model.predict(X_test)
print("...Done.")
print(Y_test_pred_ridge[0:5])
print()

# Print scores using appropriate metrics for regression
print("R-squared on training set (Ridge): ", round(r2_score(Y_train, Y_train_pred_ridge),3))
print("R-squared on test set (Ridge): ", round(r2_score(Y_test, Y_test_pred_ridge), 3))

mse_train_ridge = mean_squared_error(Y_train, Y_train_pred_ridge)
mse_test_ridge = mean_squared_error(Y_test, Y_test_pred_ridge)

print("Mean Squared Error on training set : ", round(mse_train_ridge, 2))
print("Root Mean Squared Error on training set : ", round(np.sqrt(mse_train_ridge), 2)) # Calculate and print RMSE for training set

print("Mean Squared Error on test set : ", round(mse_test_ridge, 2))
print("Root Mean Squared Error on test set : ", round(np.sqrt(mse_test_ridge), 2)) # Calculate and print RMSE for test set

Training Lasso model...
...Done.
Predictions on training set...
...Done.
[ 640707.96407186 2070134.1527097   497596.20178088 1507951.19084354
 1113133.74965071]

Predictions on test set...
...Done.
[1839124.99178736 1713586.8905849  1461711.85704495  471493.04520897
 1470017.43061295]

R-squared on training set (Ridge):  0.978
R-squared on test set (Ridge):  0.983
Mean Squared Error on training set :  10205642439.97
Root Mean Squared Error on training set :  101022.98
Mean Squared Error on test set :  7969425044.94
Root Mean Squared Error on test set :  89271.64


==> There is no more overfitting!!

In [ ]:
# Get feature names after preprocessing
feature_names = numeric_features + list(preprocessor.named_transformers_['cat']['encoder'].get_feature_names_out(categorical_features))

# Get coefficients
coefficients_ridge = ridge_model.coef_

# Create a DataFrame to display coefficients with feature names
coefficients_df_ridge = pd.DataFrame({'Feature': feature_names, 'Coefficient': coefficients_ridge})

# Sort coefficients by absolute value to see most impactful features
coefficients_df_ridge['Absolute_Coefficient'] = coefficients_df_ridge['Coefficient'].abs()
coefficients_df_ridge = coefficients_df_ridge.sort_values(by='Absolute_Coefficient', ascending=False)


# Create bar plot using Plotly
fig = px.bar(
    coefficients_df_lasso,
    x="Feature",
    y="Coefficient",
    title="Feature Coefficients of Ridge Regression Model",
)
fig.update_xaxes(tickangle=45)  # Rotate x-axis labels for readability
fig.show()

- Ridge Regression Model feature coefficents ranking is similar to Lasso Regression Model ranking.

In [ ]:
# Get the intercept (bias term)
intercept = ridge_model.intercept_

# Create the equation string
equation = f"Weekly_Sales = {intercept:.2f}"  # Start with the intercept

# Add terms for each feature with non-zero coefficients
for index, row in coefficients_df_ridge[coefficients_df_ridge['Coefficient'] != 0].iterrows():
    feature_name = row['Feature']
    coefficient = row['Coefficient']
    equation += f" + {coefficient:.2f} * {feature_name}"

# Print the equation
print("Ridge Regression Equation:")
print(equation)

Ridge Regression Equation:
Weekly_Sales = 1466037.52 + -1130225.52 * Store_5.0 + -1097231.61 * Store_3.0 + -974959.18 * Store_9.0 + -968676.30 * Store_7.0 + -872073.06 * Store_16.0 + 832151.60 * Store_4.0 + -669878.50 * Store_15.0 + -623468.87 * Store_17.0 + 604611.16 * Store_14.0 + 585497.03 * Store_13.0 + -569965.93 * Store_8.0 + 487239.26 * Store_20.0 + 439436.96 * Store_10.0 + -391991.69 * Store_18.0 + 271535.54 * Store_11.0 + 244062.30 * Store_2.0 + 172176.98 * Store_6.0 + -105857.09 * Fuel_Price + 84287.59 * Year + -80891.05 * Holiday_Flag_1.0 + -49680.00 * Day + 47192.29 * CPI + -42320.88 * Temperature + 33854.78 * Month + 17216.41 * Store_19.0 + 15301.52 * Week + 13060.93 * Unemployment


## **Part 4: Fine tuning the regularization strength**

In [ ]:
# Define the parameter grid for alpha
param_grid = {'alpha': [0.01, 0.1, 1, 10, 100]}  # You can adjust the range of alpha values

# Create Ridge model
ridge_model = Ridge()

# Create GridSearchCV object
grid_search = GridSearchCV(ridge_model, param_grid, scoring='neg_mean_squared_error', cv=5)  # cv=5 for 5-fold cross-validation

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train, Y_train)

# Get the best alpha value
best_alpha = grid_search.best_params_['alpha']
print("Best alpha:", best_alpha)

# Get the best Ridge model
best_ridge_model = grid_search.best_estimator_

# Make predictions using the best model
Y_train_pred_ridge = best_ridge_model.predict(X_train)
Y_test_pred_ridge = best_ridge_model.predict(X_test)

# Print scores using appropriate metrics for regression
print("R-squared on training set (Ridge): ", round(r2_score(Y_train, Y_train_pred_ridge), 3))
print("R-squared on test set (Ridge): ", round(r2_score(Y_test, Y_test_pred_ridge), 3))

mse_train_ridge = mean_squared_error(Y_train, Y_train_pred_ridge)
mse_test_ridge = mean_squared_error(Y_test, Y_test_pred_ridge)

print("Mean Squared Error on training set : ", round(mse_train_ridge, 2))
print("Root Mean Squared Error on training set : ", round(np.sqrt(mse_train_ridge), 2)) # Calculate and print RMSE for training set

print("Mean Squared Error on test set : ", round(mse_test_ridge, 2))
print("Root Mean Squared Error on test set : ", round(np.sqrt(mse_test_ridge), 2)) # Calculate and print RMSE for test set

Best alpha: 0.01
R-squared on training set (Ridge):  0.982
R-squared on test set (Ridge):  0.977
Mean Squared Error on training set :  8328823397.32
Root Mean Squared Error on training set :  91262.39
Mean Squared Error on test set :  10815484687.07
Root Mean Squared Error on test set :  103997.52


- Not sure we get better results with alpha = 0.01 since the test set metrics are less good.

## **Part 5 : Introduction of weights for 'Store'**

In [ ]:
# Calculate weights based on the number of observations per store
store_counts = sales_df['Store'].value_counts()
weights = sales_df['Store'].map(store_counts)

# Split data into train and test sets, including weights
X_train, X_test, Y_train, Y_test, weights_train, weights_test = train_test_split(
    X, Y, weights, test_size=0.2, random_state=0
)

# Train model with sample weights
model = LinearRegression()
model.fit(X_train, Y_train, sample_weight=weights_train)  # Include sample_weight

# Predictions on training set
Y_train_pred = model.predict(X_train)

# Predictions on test set
Y_test_pred = model.predict(X_test)

# Print scores using appropriate metrics for regression
print("R-squared on training set : ", round(r2_score(Y_train, Y_train_pred, sample_weight=weights_train), 2))
print("R-squared on test set : ", round(r2_score(Y_test, Y_test_pred, sample_weight=weights_test), 2))

mse_train = mean_squared_error(Y_train, Y_train_pred, sample_weight=weights_train)
mse_test = mean_squared_error(Y_test, Y_test_pred, sample_weight=weights_test)

print("Mean Squared Error on training set : ", round(mse_train, 2))
print("Root Mean Squared Error on training set : ", round(np.sqrt(mse_train), 2))

print("Mean Squared Error on test set : ", round(mse_test, 2))
print("Root Mean Squared Error on test set : ", round(np.sqrt(mse_test), 2))

R-squared on training set :  0.37
R-squared on test set :  0.26
Mean Squared Error on training set :  299286476023.55
Root Mean Squared Error on training set :  547070.81
Mean Squared Error on test set :  351859581866.52
Root Mean Squared Error on test set :  593177.53


- The introduction of weights based on the number of observations per store make it worse!

## **Conclusions**

*   The linear regression baseline model overfits since the number of observations is very low.
*   The best model is a linear regression model with Ridge regularization with a regularization strength of 0.1.
*   Lasso didn't help to drop potential non-impactful features (we have already dropped 'DayOfWeek').
*   Introducing weights based on the number of observations per store made the model even worse.







In [ ]:
# Create bar plot using Plotly
fig = px.bar(
    coefficients_df_lasso,
    x="Feature",
    y="Coefficient",
    title="Feature Coefficients of Ridge Regression Model",
)
fig.update_xaxes(tickangle=45)  # Rotate x-axis labels for readability
fig.show()


*   The store is the most impactful feature on the weekly sales which is expected since the surface area, the surroundings and the geographic location are important factors. Some stores have a positive impact on the weekly sales while others have a negative one such as store 5, 3 and 9. Here were Walmart should put an effort and try to understand what characteristic in a store make it more profitable.


*   The CPI (Prevailing Consumer Price Index) impacts also the weelky sales and it's related to the location.